# Clone and Compare Workflow

This notebook demonstrates the **CLB Engineering LLM Forward Approach** to QAQC:

- Non-destructive cloning of basin, met, and run configurations
- Side-by-side comparison in HEC-HMS GUI
- Parallel execution of baseline vs. updated scenarios

## Why Clone?

Cloning allows you to:
1. **Preserve original baseline models** - No risk of losing original work
2. **Test modifications safely** - Experiment without consequences
3. **Compare results side-by-side** - Visual GUI comparison
4. **Maintain traceability** - Clone metadata in descriptions
5. **Use separate DSS files** - Clean comparison between scenarios

In [ ]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

In [ ]:
from pathlib import Path
from hms_commander import (
    HmsExamples,
    init_hms_project,
    HmsBasin,
    HmsMet,
    HmsRun,
    HmsCmdr,
    HmsResults,
    HmsDss
)

print("hms-commander loaded")

## 1. Initialize Project and Extract Example

In [ ]:
# Extract HMS example project
project_path = HmsExamples.extract_project(
    "castro",
    output_path=Path.cwd() / 'hms_example_projects' / 'castro_clone_workflow'
)
print(f"Project extracted to: {project_path}")

# Initialize project
hms = init_hms_project(project_path)

# View available components
print("\nBasin Models:")
print(hms.basin_df[['name', 'total_area', 'loss_methods']])
print("\nMet Models:")
print(hms.met_df[['name', 'precip_method']])
print("\nRuns:")
print(hms.run_df[['name', 'basin_model', 'met_model', 'dss_file']])

## 2. Clone Basin Model

Clone the baseline basin to create an updated version for modification.

In [ ]:
# Clone basin for modifications
baseline_basin = hms.list_basin_names()[0]
print(f"Cloning basin: {baseline_basin}")

new_basin_path = HmsBasin.clone_basin(
    template_basin=baseline_basin,
    new_name="Castro_Updated",
    description="Updated parameters for calibration comparison",
    hms_object=hms
)

if new_basin_path:
    print(f"\n[OK] Basin cloned successfully!")
    print(f"New basin file: {new_basin_path}")
    print("New basin will appear in HEC-HMS GUI")

## 3. Modify Parameters in Cloned Basin

Update parameters in the cloned basin while leaving the original intact.

In [ ]:
# Get the cloned basin file
cloned_basin_file = project_path / "Castro_Updated.basin"

if cloned_basin_file.exists():
    # Get all subbasins
    subbasins = HmsBasin.get_subbasins(str(cloned_basin_file))
    print(f"Found {len(subbasins)} subbasins in cloned basin")
    
    # Show current parameters
    print("\nCurrent subbasin parameters:")
    print("=" * 60)
    for idx, row in subbasins.iterrows():
        subbasin_name = row['name']
        params = HmsBasin.get_loss_parameters(str(cloned_basin_file), subbasin_name)
        print(f"  {subbasin_name}: {params.get('percent_impervious', 'N/A')}% impervious")
else:
    print("Cloned basin file not found - clone may have failed")

In [ ]:
# Modify parameters (increase imperviousness by 5%)
if cloned_basin_file.exists():
    print("Updating percent impervious (+5%):")
    print("=" * 60)
    
    for idx, row in subbasins.iterrows():
        subbasin_name = row['name']
        params = HmsBasin.get_loss_parameters(str(cloned_basin_file), subbasin_name)
        
        if 'percent_impervious' in params and params['percent_impervious'] is not None:
            old_value = params['percent_impervious']
            new_value = min(old_value + 5.0, 100.0)  # Cap at 100%
            
            HmsBasin.set_loss_parameters(
                str(cloned_basin_file),
                subbasin_name,
                percent_impervious=new_value
            )
            print(f"  {subbasin_name}: {old_value}% -> {new_value}%")

## 4. Clone Meteorologic Model (Optional)

Clone the met model if you need to modify precipitation inputs.

In [ ]:
# Clone met model (optional - only if modifying precipitation)
met_names = hms.list_met_names()
print(f"Available met models: {met_names}")

if met_names:
    new_met_path = HmsMet.clone_met(
        template_met=met_names[0],
        new_name="Castro_Met_Updated",
        description="Cloned met model for comparison",
        hms_object=hms
    )
    print(f"\n[OK] Met model cloned: {new_met_path}")

## 5. Clone Run Configuration

**Critical for QAQC**: Create a new run with separate DSS output for comparison.

In [ ]:
# Clone run with updated components and separate DSS output
run_names = hms.list_run_names()
print(f"Available runs: {run_names}")

if run_names:
    source_run = run_names[0]
    
    success = HmsRun.clone_run(
        source_run=source_run,
        new_run_name="Updated_Scenario",
        new_basin="Castro_Updated",
        new_met="Castro_Met_Updated",  # Use cloned met
        output_dss="updated_scenario.dss",
        description="Increased imperviousness scenario (+5%)",
        hms_object=hms
    )
    
    if success:
        print("\n[OK] Run configuration cloned!")
        print(f"Baseline: {source_run} -> original DSS")
        print("Updated:  Updated_Scenario -> updated_scenario.dss")

## 6. Verify Both Run Configurations

Display baseline and cloned run configurations side-by-side.

In [ ]:
# Reinitialize to refresh DataFrames
hms = init_hms_project(project_path)

# Get baseline run configuration
print("Baseline Run Configuration:")
print("=" * 60)
baseline_runs = [r for r in hms.list_run_names() if 'Updated' not in r and 'Scenario' not in r]
if baseline_runs:
    try:
        baseline_config = HmsRun.get_dss_config(baseline_runs[0], hms_object=hms)
        print(f"  Run Name:   {baseline_runs[0]}")
        print(f"  Basin:      {baseline_config.get('basin_model', 'N/A')}")
        print(f"  Met:        {baseline_config.get('met_model', 'N/A')}")
        print(f"  DSS Output: {baseline_config.get('dss_file', 'N/A')}")
    except Exception as e:
        print(f"  Could not get config: {e}")

# Get cloned run configuration
print("\nCloned Run Configuration:")
print("=" * 60)
cloned_runs = [r for r in hms.list_run_names() if 'Updated' in r or 'Scenario' in r]
if cloned_runs:
    try:
        cloned_config = HmsRun.get_dss_config(cloned_runs[0], hms_object=hms)
        print(f"  Run Name:   {cloned_runs[0]}")
        print(f"  Basin:      {cloned_config.get('basin_model', 'N/A')}")
        print(f"  Met:        {cloned_config.get('met_model', 'N/A')}")
        print(f"  DSS Output: {cloned_config.get('dss_file', 'N/A')}")
        print(f"  Description: {cloned_config.get('description', 'N/A')}")
    except Exception as e:
        print(f"  Could not get config: {e}")
else:
    print("  No cloned runs found")

## 7. Execute Both Runs

Run both baseline and updated scenarios to generate results for comparison.

**Note**: Execution requires HEC-HMS to be installed and properly configured.

In [ ]:
# Execute baseline run
if baseline_runs:
    print(f"Executing baseline run: {baseline_runs[0]}")
    try:
        result = HmsCmdr.compute_run(baseline_runs[0], hms_object=hms)
        # Handle both boolean and result object returns
        if hasattr(result, 'success'):
            if result.success:
                print(f"  [OK] Completed in {result.execution_time:.1f}s")
            else:
                print(f"  [FAILED] {result.error_message}")
        elif result:  # Boolean True
            print("  [OK] Execution completed")
        else:
            print("  [FAILED] Execution returned False")
    except Exception as e:
        print(f"  [SKIP] Execution not available: {e}")

In [ ]:
# Execute cloned run
if cloned_runs:
    print(f"Executing cloned run: {cloned_runs[0]}")
    try:
        result = HmsCmdr.compute_run(cloned_runs[0], hms_object=hms)
        # Handle both boolean and result object returns
        if hasattr(result, 'success'):
            if result.success:
                print(f"  [OK] Completed in {result.execution_time:.1f}s")
            else:
                print(f"  [FAILED] {result.error_message}")
        elif result:  # Boolean True
            print("  [OK] Execution completed")
        else:
            print("  [FAILED] Execution returned False")
    except Exception as e:
        print(f"  [SKIP] Execution not available: {e}")

## 8. Compare Results

Compare peak flows and hydrographs between baseline and updated scenarios.

**Note**: This section requires DSS files to exist from executed runs.

In [ ]:
# Get DSS file paths
baseline_dss = None
updated_dss = project_path / "updated_scenario.dss"

# Determine baseline DSS path from run configuration
if baseline_runs:
    try:
        baseline_config = HmsRun.get_dss_config(baseline_runs[0], hms_object=hms)
        baseline_dss = project_path / baseline_config.get('dss_file', 'castro.dss')
    except Exception:
        baseline_dss = project_path / "castro.dss"

print("=== Results Comparison ===")
print(f"Baseline DSS: {baseline_dss}")
print(f"  Exists: {baseline_dss.exists() if baseline_dss else False}")
print(f"Updated DSS: {updated_dss}")
print(f"  Exists: {updated_dss.exists()}")

In [ ]:
# Compare peak flows if both DSS files exist
dss_available = HmsDss.is_available()
baseline_exists = baseline_dss and baseline_dss.exists()
updated_exists = updated_dss.exists()

print(f"DSS library available: {dss_available}")
print(f"Both DSS files exist: {baseline_exists and updated_exists}")

if dss_available and baseline_exists and updated_exists:
    print("\n=== Peak Flow Comparison ===")
    try:
        # Get peak flows from both runs
        baseline_peaks = HmsResults.get_peak_flows(str(baseline_dss))
        print("\nBaseline Peak Flows:")
        print(baseline_peaks)
    except Exception as e:
        print(f"Could not read baseline peaks: {e}")
    
    try:
        updated_peaks = HmsResults.get_peak_flows(str(updated_dss))
        print("\nUpdated Peak Flows:")
        print(updated_peaks)
    except Exception as e:
        print(f"Could not read updated peaks: {e}")
else:
    print("\n[INFO] DSS comparison not available.")
    if not dss_available:
        print("  - DSS library (pyjnius/Java) not configured")
    if not baseline_exists:
        print("  - Baseline DSS file does not exist")
    if not updated_exists:
        print("  - Updated DSS file does not exist")
    print("\nRun HEC-HMS simulations first to generate DSS output files.")

## 9. Visualize Comparison (Optional)

Generate hydrograph overlay if DSS files and matplotlib are available.

In [ ]:
# Visualization of results (requires matplotlib and DSS files)
try:
    import matplotlib.pyplot as plt
    matplotlib_available = True
except ImportError:
    matplotlib_available = False
    print("[INFO] matplotlib not installed - visualization skipped")

if matplotlib_available and dss_available and baseline_exists and updated_exists:
    print("Generating hydrograph comparison...")
    try:
        # Get catalog to find flow paths
        baseline_catalog = HmsDss.get_catalog(str(baseline_dss))
        flow_paths = [p for p in baseline_catalog if 'FLOW' in p.upper() and 'OBSERVED' not in p.upper()]
        
        if flow_paths:
            # Read first flow path from both DSS files
            pathname = flow_paths[0]
            print(f"Reading pathname: {pathname}")
            
            try:
                baseline_ts = HmsDss.read_timeseries(str(baseline_dss), pathname)
                updated_ts = HmsDss.read_timeseries(str(updated_dss), pathname)
                
                # Plot comparison
                fig, ax = plt.subplots(figsize=(12, 6))
                ax.plot(baseline_ts.index, baseline_ts['value'], label='Baseline', linewidth=2)
                ax.plot(updated_ts.index, updated_ts['value'], label='Updated (+5% Imp.)', 
                         linewidth=2, linestyle='--')
                ax.set_xlabel('Time')
                ax.set_ylabel('Flow (cfs)')
                ax.set_title('Hydrograph Comparison: Baseline vs Updated')
                ax.legend()
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
            except TypeError as e:
                # Handle the "len() of unsized object" error from DSS read
                print(f"[WARNING] Could not read DSS time series: {e}")
                print("  This may indicate empty or malformed DSS data.")
            except Exception as e:
                print(f"[WARNING] Could not plot hydrographs: {e}")
        else:
            print("[INFO] No FLOW paths found in DSS catalog")
    except Exception as e:
        print(f"[WARNING] Could not generate visualization: {e}")
elif matplotlib_available:
    print("[INFO] Visualization skipped - DSS files not available")
    print("Run HEC-HMS simulations to generate results for comparison.")

## 10. Verify in HEC-HMS GUI

After running this notebook, open the project in HEC-HMS GUI:

1. You should see:
   - Original basin: `Castro 1`
   - Cloned basin: `Castro_Updated`
   - Original run: `Current`
   - Cloned run: `Updated_Scenario`

2. Compare results:
   - Open both result plots side-by-side
   - Check descriptions for clone metadata
   - Verify parameter changes in basin editor

## Summary

This workflow demonstrates the **CLB Engineering LLM Forward** approach:

| Principle | Implementation |
|-----------|---------------|
| **Non-destructive** | Original models preserved via cloning |
| **Traceable** | Clone metadata in descriptions |
| **GUI-verifiable** | Components visible in HEC-HMS |
| **Efficient** | Parallel execution capability |
| **Clean comparison** | Separate DSS outputs |

**Perfect for**:
- Parameter sensitivity analysis
- Model updates and calibration
- QAQC workflows
- Scenario comparison

## Next Steps

- **06_results_dss.ipynb**: Deep dive into DSS operations and results extraction
- **07_execution_jython.ipynb**: Advanced execution patterns
- **04_run_management.ipynb**: More details on run configuration